In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/gdrive')

In [0]:
!bash /gdrive/My\ Drive/Miniconda3-latest-Linux-x86_64.sh
!/root/miniconda3/bin/conda install pytorch torchvision -c pytorch

In [0]:
!root/miniconda3/bin/conda create -n pysyft python=3
!root/miniconda3/bin/activate pysyft # some older version of conda require "source activate pysyft" instead.
!root/miniconda3/bin/conda install jupyter notebook
!pip install syft
!pip install numpy

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [6]:
!pip install syft

     |████████████████████████████████| 215kB 4.8MB/s 
     |████████████████████████████████| 81kB 24.1MB/s 
     |████████████████████████████████| 1.4MB 44.9MB/s 
     |████████████████████████████████| 204kB 44.2MB/s 
     |████████████████████████████████| 389kB 44.7MB/s 
     |████████████████████████████████| 256kB 46.2MB/s 
     |████████████████████████████████| 450kB 45.9MB/s 
     |████████████████████████████████| 51kB 19.0MB/s 
     |████████████████████████████████| 276kB 41.8MB/s 
     |████████████████████████████████| 122kB 42.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [0]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [0]:
import syft as sy

In [0]:
hook = sy.TorchHook(th)

In [10]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [12]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [15]:
bob._objects

{60574846680: tensor([1, 2, 3, 4, 5])}

In [16]:
x.location

<VirtualWorker id:bob #objects:1>

In [17]:
x.id_at_location

60574846680

In [18]:
x.id

39060533449

In [19]:
x.owner

<VirtualWorker id:me #objects:0>

In [20]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [21]:
x

(Wrapper)>[PointerTensor | me:39060533449 -> bob:60574846680]

In [22]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [23]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# try this project here!

In [0]:
ashley = sy.VirtualWorker(hook, id='ashley')
meghan = sy.VirtualWorker(hook, id='meghan')

In [0]:
ten = th.tensor([1,2,3,4,5])

In [0]:
ten=ten.send(ashley, meghan)

In [86]:
ten

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:73116593991 -> ashley:1560404622]
	-> (Wrapper)>[PointerTensor | me:58762547491 -> meghan:21644137593]

In [87]:
ashley._objects, meghan._objects

({1560404622: tensor([1, 2, 3, 4, 5])}, {21644137593: tensor([1, 2, 3, 4, 5])})

In [0]:
ten = ten.get()

In [89]:
ten

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [26]:
x

(Wrapper)>[PointerTensor | me:51612903956 -> bob:2383555618]

In [27]:
y

(Wrapper)>[PointerTensor | me:51581218933 -> bob:62366869740]

In [0]:
z = x + y

In [29]:
z

(Wrapper)>[PointerTensor | me:90557805694 -> bob:73753395458]

In [30]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [31]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:63077543148 -> bob:91033192703]

In [32]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [35]:
z.backward()

(Wrapper)>[PointerTensor | me:8453924600 -> bob:45017355280]

In [0]:
x = x.get()

In [37]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [38]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
# try this project here!

# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [41]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [43]:
bob._objects

{61317185591: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [45]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{2429142483: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [49]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
x

(Wrapper)>[PointerTensor | me:84271318942 -> bob:66521954497]

In [52]:
bob._objects

{66521954497: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [54]:
bob._objects

{66521954497: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [56]:
bob._objects

{66521954497: tensor([1, 2, 3, 4, 5])}

In [57]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [59]:
bob._objects

{56180930774: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [61]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x + y

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [0]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [0]:
train()

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [0]:
bob.clear_objects()
alice.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
y = x + x

In [0]:
y

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
x = x.get()
x

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
x = x.get()
x

In [0]:
bob._objects

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
del x

In [0]:
bob._objects

In [0]:
alice._objects

# Lesson: Pointer Chain Operations

In [0]:
bob.clear_objects()
alice.clear_objects()

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
x.move(alice)

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
x.remote_get()

In [0]:
bob._objects

In [0]:
alice._objects

In [0]:
x.move(bob)

In [0]:
x

In [0]:
bob._objects

In [0]:
alice._objects